In [0]:
# Databricks Widgets Setup
dbutils.widgets.combobox("Environment", "", ["dev", "prod"])
dbutils.widgets.text("CatalogName", "_nyctaxi_catalog")
dbutils.widgets.combobox("SchemaName", "",["bronze", "silver", "gold"])
dbutils.widgets.combobox("StoragePath", "",["landing_path","bronze_path", "silver_path", "gold_path"])

# Get widget values
Environment = dbutils.widgets.get("Environment")
catalog_name = dbutils.widgets.get("CatalogName")
schema_name = dbutils.widgets.get("SchemaName")
storage_path = dbutils.widgets.get("StoragePath")


In [0]:
#dbutils.widgets.removeAll()

In [0]:
landing_path = spark.sql("""DESCRIBE EXTERNAL LOCATION landingzone_external_location""").select("url").collect()[0][0]
bronze_path = spark.sql("""DESCRIBE EXTERNAL LOCATION bronze_external_location""").select("url").collect()[0][0]
silver_path = spark.sql("""DESCRIBE EXTERNAL LOCATION silver_external_location""").select("url").collect()[0][0]
gold_path = spark.sql("""DESCRIBE EXTERNAL LOCATION gold_external_location""").select("url").collect()[0][0]

print(bronze_path)
print(silver_path)
print(gold_path)
print(landing_path)

path_mapping = {
        "landing_path": landing_path,
        "bronze_path": bronze_path,
        "silver_path": silver_path,
        "gold_path": gold_path
    }
resolved_path = path_mapping.get(storage_path)
print(resolved_path)    
if resolved_path is None:
    raise ValueError(f"Invalid storage_path widget value: {storage_path}")


In [0]:
def create_catalog(Environment, catalog_name):
    try:
        full_catalog_name = f"{Environment}{catalog_name}"
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {full_catalog_name}")
        print(f"Catalog {full_catalog_name} is created successfully.")
    except Exception as e:
        print(f"Catalog {full_catalog_name} is not created. {e}")
           

In [0]:
def create_schema(Environment, catalog_name, schema_name, storage_path):
    full_catalog_name = f"{Environment}{catalog_name}"
    try:
        spark.sql(f"USE CATALOG {full_catalog_name}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{schema_name}` MANAGED LOCATION '{resolved_path}/{schema_name}_files'")
        print(f"Schema {schema_name} is created successfully.")
    except Exception as e:
        print(f"Schema {schema_name} is not created. {e}")



In [0]:
create_catalog(Environment, catalog_name)
create_schema(Environment, catalog_name, schema_name, storage_path)

In [0]:
 schema_check = spark.sql(f"SHOW SCHEMAS LIKE '{schema_name}'").collect()
 print(schema_check)